## Import necessary modules and data

In [1]:
import numpy as np
import pandas as pd
import sklearn
import keras
import seaborn
import matplotlib.pyplot as plt
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
#!pip install -q keras_metrics
import keras_metrics
from sklearn.model_selection import cross_validate
#!pip install livelossplot
from livelossplot import PlotLossesKeras
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.metrics import precision_recall_fscore_support as score
import random


Using TensorFlow backend.


## Wczytujemy poprzednio utworzone zbiory

In [2]:
X = pd.read_csv('../data/cleaned_data.csv', index_col=0)
y = pd.read_csv('../data/labels.csv', header = None)

X_test = pd.read_csv('../data/X_test.csv', index_col=0)
X_train = pd.read_csv('../data/X_train.csv', index_col=0)
X_train_std = pd.read_csv('../data/X_train_std.csv', index_col=0)
X_test_std = pd.read_csv('../data/X_test_std.csv', index_col=0)
y_train = pd.read_csv('../data/y_train.csv', index_col=0)
y_test = pd.read_csv('../data/y_test.csv', index_col=0)

Zbuduję przykładową sieć, która składa się z warstwy wejściowej, jednej warstwy ukrytej i  warstwy wyjściowej.
Kompilujemy model przy użyciu 

loss_function='binary_crossentropy';

optimizer='Adam'.


Naszą główną metryką będzie **precyzja** oznaczająca procent osób prawidłowie określonych jako zagrożone odejściem.

## Pierwsze podejście do modelu

In [3]:
model = Sequential()

model.add(Dense(11, activation='relu', input_shape=(10,)))
 
model.add(Dense(121, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[keras_metrics.precision(),'accuracy'])
                   
model.fit(X_train_std, y_train,epochs=3, batch_size=1, verbose=1, validation_data=(X_test,y_test))

model.evaluate(X_test_std,y_test)

Train on 6700 samples, validate on 3300 samples
Epoch 1/3
6700/6700 [==============================] - 19s 3ms/step - loss: 0.4307 - precision: 0.7170 - acc: 0.8237 - val_loss: 12.6759 - val_precision: 0.2039 - val_acc: 0.2048
Epoch 2/3
6700/6700 [==============================] - 19s 3ms/step - loss: 0.3651 - precision: 0.7437 - acc: 0.8513 - val_loss: 3.7012 - val_precision: 0.2442 - val_acc: 0.7697
Epoch 3/3
3300/3300 [==============================] - 0s 34us/step


[0.3568253916321379, 0.7388888886836419, 0.8484848484848485]

## Sprawdźmy skuteczność naszej predykcji raportem klasyfikacyjnym


In [4]:
y_pred = model.predict_classes(X_test_std)
report = classification_report(y_test,y_pred)
report1 = classification_report(y_test,y_pred,output_dict=True)
print(report)
print(report1['0']['precision'])


              precision    recall  f1-score   support

           0       0.86      0.96      0.91      2628
           1       0.74      0.40      0.52       672

   micro avg       0.85      0.85      0.85      3300
   macro avg       0.80      0.68      0.71      3300
weighted avg       0.84      0.85      0.83      3300

0.861904761904762


### Sprawdźmy, jak prezentują się wyniki w zależności od liczby neuronów w warstwie ukrytej.

In [5]:
nodes = [10, 30, 50, 70, 90, 110]
wyniki = {}
for i in nodes:
    model = 0

    model = Sequential()

    model.add(Dense(11, activation='relu', input_shape=(10,)))

    model.add(Dense(i, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=[keras_metrics.precision(), 'accuracy'])

    model.fit(X_train_std, y_train, epochs=5, batch_size=1, verbose=1, validation_data=(X_test, y_test))
    print("Zakończono trenowanie modelu z {0} węzłami.".format(i))
    model.evaluate(X_test_std, y_test)
    y_pred = model.predict_classes(X_test_std)
    report = classification_report(y_test, y_pred, output_dict=True)
    wyniki.update({i: [report['0']['precision'], report['1']['precision']]})

print(wyniki)


Train on 6700 samples, validate on 3300 samples
Epoch 1/5
6700/6700 [==============================] - 19s 3ms/step - loss: 0.4513 - precision: 0.6040 - acc: 0.8087 - val_loss: 7.9938 - val_precision: 0.1584 - val_acc: 0.4997
Epoch 2/5
6700/6700 [==============================] - 20s 3ms/step - loss: 0.3946 - precision: 0.7204 - acc: 0.8375 - val_loss: 3.2822 - val_precision: 0.0000e+00 - val_acc: 0.7964
Epoch 3/5
6700/6700 [==============================] - 23s 3ms/step - loss: 0.3616 - precision: 0.7625 - acc: 0.8557 - val_loss: 3.2773 - val_precision: 1.0000 - val_acc: 0.7967
Epoch 4/5
6700/6700 [==============================] - 22s 3ms/step - loss: 0.3533 - precision: 0.7621 - acc: 0.8578 - val_loss: 4.3597 - val_precision: 0.2075 - val_acc: 0.7255
Epoch 5/5
6700/6700 [==============================] - 21s 3ms/step - loss: 0.3496 - precision: 0.7734 - acc: 0.8600 - val_loss: 10.7266 - val_precision: 0.1863 - val_acc: 0.3279
Zakończono trenowanie modelu z 10 węzłami.
3300/3300 [===

## Wybieramy model z 90 węzłami - dał najlepsze wyniki

In [6]:
from keras.callbacks import EarlyStopping

model = Sequential()

model.add(Dense(10, activation='relu', input_shape=(10,)))
model.add(Dropout(0.1, ))

model.add(Dense(90, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[keras_metrics.precision(), 'accuracy'])

early_stopping_monitor = EarlyStopping(patience=2)
model.fit(X_train_std, y_train, epochs=30, batch_size=1, verbose=1, validation_data=(X_test, y_test),
          callbacks=[early_stopping_monitor])

model.evaluate(X_test_std, y_test)
y_pred = model.predict_classes(X_test_std)
report = classification_report(y_test, y_pred)
report1 = classification_report(y_test, y_pred, output_dict=True)
print(report)



Train on 6700 samples, validate on 3300 samples
Epoch 1/30
6700/6700 [==============================] - 22s 3ms/step - loss: 0.4308 - precision: 0.6687 - acc: 0.8204 - val_loss: 3.2822 - val_precision: 0.0000e+00 - val_acc: 0.7964
Epoch 2/30
6700/6700 [==============================] - 23s 3ms/step - loss: 0.3834 - precision: 0.7049 - acc: 0.8410 - val_loss: 3.2822 - val_precision: 0.0000e+00 - val_acc: 0.7964
Epoch 3/30
3300/3300 [==============================] - 0s 38us/step
              precision    recall  f1-score   support

           0       0.87      0.96      0.91      2628
           1       0.73      0.42      0.53       672

   micro avg       0.85      0.85      0.85      3300
   macro avg       0.80      0.69      0.72      3300
weighted avg       0.84      0.85      0.83      3300



## Zapisujemy wytrenowany model

In [8]:
model.save("../saved_models/model_1.h5")

### autor: Tomasz Sołtysiak